In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import teneva
np.random.seed(42)

In [2]:
a = -1.
b = +1.
d = 5
n = 10
m = 1.E+5
r = 4
q = 8

In [3]:
I_trn = teneva.sample_lhs([n]*d, m)
I_tst = teneva.sample_lhs([n]*d, m)

X_trn = teneva.ind_to_poi(I_trn, a, b, n, 'cheb')
X_tst = teneva.ind_to_poi(I_tst, a, b, n, 'cheb')

Y_trn = sp.optimize.rosen(X_trn.T)
Y_tst = sp.optimize.rosen(X_tst.T)

In [4]:
Y0 = teneva.tensor_rand([q]*d, r)
teneva.show(Y0)

TT-tensor     5D : |8| |8| |8| |8| |8|
<rank>  =    4.0 :   \4/ \4/ \4/ \4/


In [5]:
f = lambda X: teneva.cheb_pol(X, a, b, q).T
A = teneva.als_spectral(X_trn, Y_trn, Y0, f, log=True, nswp=10)

# pre | time:      0.063 | rank:   4.0 | 
#   1 | time:      4.860 | rank:   4.0 | eps: 1.3e+00 | 
#   2 | time:      8.230 | rank:   4.0 | eps: 1.0e-01 | 
#   3 | time:     11.930 | rank:   4.0 | eps: 2.0e-04 | 
#   4 | time:     15.922 | rank:   4.0 | eps: 7.2e-07 | 
#   5 | time:     19.937 | rank:   4.0 | eps: 0.0e+00 | stop: e | 


In [5]:
A = teneva.als_spectral_cheb(X_trn, Y_trn, Y0, a, b, nswp=10, log=True)

# pre | time:      0.078 | rank:   4.0 | 
#   1 | time:      6.846 | rank:   4.0 | eps: 1.0e+00 | 
#   2 | time:     12.224 | rank:   4.0 | eps: 1.4e-01 | 
#   3 | time:     17.611 | rank:   4.0 | eps: 5.0e-04 | 
#   4 | time:     23.076 | rank:   4.0 | eps: 2.6e-06 | 
#   5 | time:     28.854 | rank:   4.0 | eps: 0.0e+00 | stop: e | 


In [6]:
Y_trn_appr = teneva.cheb_get(X_trn, A, a, b)

np.linalg.norm(Y_trn_appr - Y_trn) / np.linalg.norm(Y_trn)

1.1270514591387253e-10

In [7]:
Y_tst_appr = teneva.cheb_get(X_tst, A, a, b)

np.linalg.norm(Y_tst_appr - Y_tst) / np.linalg.norm(Y_tst)

1.145845511391676e-10

In [8]:
from time import perf_counter as tpc

In [10]:
for func in teneva.func_demo_all(d=7, with_piston=True):
    func.build_trn_ind(1.E+5)
    func.build_tst_poi(1.E+5)
    
    func.method = func.method + '-ALS_SPEC' if func.method else 'ALS_SPEC'

    t = tpc()
    q = 3
    A0 = teneva.tensor_rand([q]*func.d, r=4)
    func.A = teneva.als_spectral_cheb(func.X_trn_ind, func.Y_trn_ind, A0,
        func.a, func.b, nswp=10, log=True)
    func.t += tpc() - t

    if not func.m:
        func.m = func.m_trn_ind

    func.Y = teneva.cheb.gets(func.A, func.a, func.b, func.n)
    func.r = teneva.erank(func.Y)

    func._get = teneva.getter(func.Y)


ValueError: Invalid grid option